Author: Owen Burt  
Date created: 2/21/2025  
Description:  
References:  
Sources:  

In [2]:
import tensorflow as tf
import pandas as pd
import cv2
import os as os
import glob

### Creating training and testing data
I need to convert my images to arrays and make an array for labels and bounds.  
The bounds will be combined with labels creating the target.
I need to somehow create training and testing splits in the data.  
Also I need to form a dataset with these arrays.  
What does that look like????

If i were to do classification I would need a seperate model. So one to do bounds and one to do class.  
I think I will just detect the bounds for this project.  
steps are to flatten images (maybe also shrink them) split the image data and bound data into training and testing.  
build model.

In [3]:
cwd = os.getcwd()
print(cwd)

c:\Users\owend\Projects\Eye_tracker


In [4]:
df = pd.read_csv(cwd + "/eye_data.csv")

In [5]:
df

,file_name,obj_class,x1,y1,x2,y2
0,WIN_20250205_16_00_55_Pro.jpg,pupil,257,163,276,195
1,WIN_20250205_16_00_56_Pro.jpg,pupil,279,128,308,156
2,WIN_20250205_16_00_57_Pro.jpg,pupil,313,138,346,170
3,WIN_20250205_16_00_58_Pro.jpg,pupil,278,141,305,172
4,WIN_20250205_16_00_59_Pro.jpg,pupil,263,150,286,181
...,...,...,...,...,...,...
363,WIN_20250220_20_01_15_Pro.jpg,NEGATIVE,0,0,0,0
364,WIN_20250220_20_01_17_Pro.jpg,NEGATIVE,0,0,0,0
365,WIN_20250220_20_01_19_Pro.jpg,NEGATIVE,0,0,0,0
366,WIN_20250220_20_01_28_Pro.jpg,NEGATIVE,0,0,0,0


### Splitting data into training, testing, and validation groups.

In [6]:
from sklearn.model_selection import train_test_split

X = df['file_name']
y = df[['x1', 'y1', 'x2', 'y2']]

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

print(x_train, x_val, x_test)

312    WIN_20250208_18_45_49_Pro.jpg
98     WIN_20250205_16_22_53_Pro.jpg
353    WIN_20250220_20_00_37_Pro.jpg
53     WIN_20250205_16_21_48_Pro.jpg
345    WIN_20250220_19_59_22_Pro.jpg
                   ...              
146    WIN_20250208_18_40_55_Pro.jpg
177    WIN_20250208_18_41_32_Pro.jpg
8      WIN_20250205_16_20_49_Pro.jpg
131    WIN_20250208_18_40_38_Pro.jpg
240    WIN_20250208_18_44_05_Pro.jpg
Name: file_name, Length: 276, dtype: object 295    WIN_20250208_18_45_23_Pro.jpg
57     WIN_20250205_16_21_54_Pro.jpg
129    WIN_20250208_18_40_35_Pro.jpg
351    WIN_20250220_19_59_50_Pro.jpg
266    WIN_20250208_18_44_41_Pro.jpg
284    WIN_20250208_18_45_07_Pro.jpg
136    WIN_20250208_18_40_43_Pro.jpg
252    WIN_20250208_18_44_22_Pro.jpg
148    WIN_20250208_18_40_58_Pro.jpg
364    WIN_20250220_20_01_17_Pro.jpg
181    WIN_20250208_18_41_41_Pro.jpg
211    WIN_20250208_18_43_30_Pro.jpg
197    WIN_20250208_18_41_58_Pro.jpg
234    WIN_20250208_18_43_58_Pro.jpg
342    WIN_20250220_19_59_18_Pr

Converting pixel data to lists

In [ ]:
#ran into an issue here and need to combine all image files into one folder

from tensorflow.keras.preprocessing import image

def flatten_img(X, y):
    image_list_name = []
    boundry_list_name = []
    for row in X:
        image_path = os.getcwd + ''

In [7]:
#from tensorflow.keras.preprocessing import image